In [ ]:
# default_exp models

# Models

> Unet models.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export 
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from functools import partial

In [ ]:
#export
def Unet2D(n_channels=1, n_classes=2, n_levels=4, n_features=64,
           n_batch_norm_levels=0, upsample = False,
           relu_alpha=0.1, k_init="he_normal"):

    # Define custom layers
    conf2d = partial(layers.Conv2D, padding="valid", kernel_initializer= k_init)
    conf2dT = partial(layers.Conv2DTranspose, padding="valid", kernel_initializer= k_init)

    # Inputs
    inputs = layers.Input(shape=(None, None,  n_channels), name="data")
    down_stack = []

    # Modules of the analysis path consist of two convolutions and max pooling
    for l in range( n_levels):
        x = conf2d(2**l *  n_features, 3, name="conv_d{}a-b".format(l))(inputs if l == 0 else x)
        x = layers.LeakyReLU(alpha= relu_alpha)(x)
        if l >  n_batch_norm_levels:
            x = layers.BatchNormalization(axis=-1)(x)
        x = conf2d(2**l *  n_features, 3, name="conv_d{}b-c".format(l))(x)
        x = layers.LeakyReLU(alpha= relu_alpha)(x)
        if l >  n_batch_norm_levels:
            x = layers.BatchNormalization(axis=-1)(x)
        if l >= 2:
            x = layers.Dropout(0.5)(x)
        down_stack.append(x)
        x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Deepest layer has two convolutions only
    x = conf2d(2** n_levels *  n_features, 3, name="conv_d{}a-b".format( n_levels))(x)
    x = layers.LeakyReLU(alpha= relu_alpha)(x)
    if l >  n_batch_norm_levels:
        x = layers.BatchNormalization(axis=-1)(x)
    x = conf2d(2** n_levels *  n_features, 3, name="conv_d{}b-c".format( n_levels))(x)
    x = layers.LeakyReLU(alpha= relu_alpha)(x)
    if l >  n_batch_norm_levels:
        x = layers.BatchNormalization(axis=-1)(x)
    pad = 8

    # Modules in the synthesis path consist of up-convolution,
    # concatenation and two convolutions
    for l in range( n_levels - 1, -1, -1):
        name = "upconv_{}{}{}_u{}a".format(
            *(("d", l+1, "c", l) if l ==  n_levels - 1 else ("u", l+1, "d", l)))
        if  upsample:
            x = layers.UpSampling2D(size=(2, 2), name=name)(x)
        else:
            x = conf2dT(2**np.max((l, 1)) *  n_features, (2, 2), strides=2, name=name)(x)
            x = layers.LeakyReLU(alpha= relu_alpha)(x)
            if l >  n_batch_norm_levels:
                x = layers.BatchNormalization(axis=-1)(x)
        x = layers.Concatenate()([layers.Cropping2D(cropping=int(pad / 2))(down_stack[l]), x])
        x = layers.LeakyReLU(alpha= relu_alpha)(x)
        x = conf2d(2**np.max((l, 1)) *  n_features, 3, name="conv_u{}b-c".format(l))(x)
        if l >  n_batch_norm_levels:
            x = layers.BatchNormalization(axis=-1)(x)
        x = conf2d(2**np.max((l, 1)) *  n_features, 3, name="conv_u{}c-d".format(l))(x)
        x = layers.LeakyReLU(alpha= relu_alpha)(x)
        if l >  n_batch_norm_levels:
            x = layers.BatchNormalization(axis=-1)(x)
        pad = 2 * (pad + 8)

    score = conf2d( n_classes, 1, name="conv_u0d-score")(x)
    softmax_score = layers.Softmax(name='softmax')(score)

    model= tf.keras.Model(inputs=inputs, outputs=[score, softmax_score])

    model.n_channels = n_channels
    model.n_levels = n_levels
    model.n_classes = n_classes
    model.n_features = n_features
    model.n_batch_norm_levels = n_batch_norm_levels
    model.relu_alpha = relu_alpha
    model.k_init = k_init
    model.upsample = upsample

    return model